# Ejercicio 4 

In [63]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [64]:
# OJO a los espacios en las columnas a mala gana!!!

b = pd.read_csv("bookings.csv.bz2", 
                sep='^', 
                usecols= ['brd_time           ', 'brd_port', 'off_port'],
                parse_dates=['brd_time           '],
                nrows=1e5)

In [65]:
b.columns

Index([u'brd_port', u'off_port', u'brd_time           '], dtype='object')

In [66]:
b.head()

,brd_port,off_port,brd_time
0,LHR,ZRH,2013-03-07 08:50:00
1,SAL,CLT,2013-04-12 13:04:00
2,CLT,SAL,2013-07-15 07:00:00
3,AKL,SVO,2013-04-24 23:59:00
4,SVO,AKL,2013-05-14 20:15:00


In [67]:
b['date'] = b['brd_time           ']

In [68]:
b = b.drop('brd_time           ', axis=1)

In [69]:
b.head()

,brd_port,off_port,date
0,LHR,ZRH,2013-03-07 08:50:00
1,SAL,CLT,2013-04-12 13:04:00
2,CLT,SAL,2013-07-15 07:00:00
3,AKL,SVO,2013-04-24 23:59:00
4,SVO,AKL,2013-05-14 20:15:00


In [70]:
# cargamos el otro fichero para hacer el matching

s = pd.read_csv("searches.csv.bz2", 
                sep='^', 
                nrows=1e5,
                usecols= ['Seg1Date', 'Origin', 'Destination'],
                parse_dates=['Seg1Date'])

In [71]:
s.columns

Index([u'Origin', u'Destination', u'Seg1Date'], dtype='object')

In [72]:
s.head()

,Origin,Destination,Seg1Date
0,TXL,AUH,2013-01-26
1,ATH,MIL,2013-01-04
2,ICT,SFO,2013-08-02
3,RNB,ARN,2013-01-02
4,OSL,MAD,2013-03-22


In [73]:
# con función
def onlyDate(d):
    return d.date()

c = b['date'].apply(onlyDate)
d = s['Seg1Date'].apply(onlyDate)
b['fecha'] = c
s['fecha'] = d

In [74]:
b.head()

,brd_port,off_port,date,fecha
0,LHR,ZRH,2013-03-07 08:50:00,2013-03-07
1,SAL,CLT,2013-04-12 13:04:00,2013-04-12
2,CLT,SAL,2013-07-15 07:00:00,2013-07-15
3,AKL,SVO,2013-04-24 23:59:00,2013-04-24
4,SVO,AKL,2013-05-14 20:15:00,2013-05-14


In [75]:
b = b.drop('date', axis=1)

In [76]:
b.head()

,brd_port,off_port,fecha
0,LHR,ZRH,2013-03-07
1,SAL,CLT,2013-04-12
2,CLT,SAL,2013-07-15
3,AKL,SVO,2013-04-24
4,SVO,AKL,2013-05-14


In [83]:
# el drop para quitar los valores que no hay. Y ojo porque hay espacios en los nombres de aeropuertos. Quitarlos!!!
merged_inner = pd.merge(s.dropna(),
                        b.dropna(), 
                        how='left',
                        left_on=['Origin', 'Destination','fecha'], 
                        right_on=['brd_port','off_port','fecha'])

In [84]:
# OJO porque hay aeropuertos que tienen códigos de ciudad.... Mirar Con GeoBasis.

merged_inner.head()

,Origin,Destination,Seg1Date,fecha,brd_port,off_port
0,TXL,AUH,2013-01-26,2013-01-26,NaN,NaN
1,ATH,MIL,2013-01-04,2013-01-04,NaN,NaN
2,ICT,SFO,2013-08-02,2013-08-02,NaN,NaN
3,RNB,ARN,2013-01-02,2013-01-02,NaN,NaN
4,OSL,MAD,2013-03-22,2013-03-22,NaN,NaN


In [85]:
merged_inner['match'] = merged_inner.brd_port.apply(lambda b: not pd.isnull(b))

In [86]:
merged_inner.head()

,Origin,Destination,Seg1Date,fecha,brd_port,off_port,match
0,TXL,AUH,2013-01-26,2013-01-26,NaN,NaN,False
1,ATH,MIL,2013-01-04,2013-01-04,NaN,NaN,False
2,ICT,SFO,2013-08-02,2013-08-02,NaN,NaN,False
3,RNB,ARN,2013-01-02,2013-01-02,NaN,NaN,False
4,OSL,MAD,2013-03-22,2013-03-22,NaN,NaN,False


In [87]:
# contamos o agrupamos para ver los que cruzan.
# también vemos si están los datos correctos o faltan datos.
merged_inner.groupby('match').count()

,Origin,Destination,Seg1Date,fecha,brd_port,off_port
match,,,,,,
False,99605,99605,99605,99605,0,0


In [ ]:
# Usaríamos una regresión logística para cruzar datos. En vez de merge que es más simple. Mirar.
# Quitar como ejercicio, los espacios en las diferentes columnas.